In [1]:
import fitz  # PyMuPDF
import os

def extract_text_from_pdfs(pdf_folder):
    texts = []
    for pdf_file in os.listdir(pdf_folder):
        if pdf_file.endswith('.pdf'):
            doc = fitz.open(os.path.join(pdf_folder, pdf_file))
            text = ""
            for page in doc:
                text += page.get_text()
            texts.append(text)
    return texts

pdf_folder = 'D:\SIH_24\law_data'
texts = extract_text_from_pdfs(pdf_folder)


In [2]:
from datasets import Dataset

data = {'text': texts}
dataset = Dataset.from_dict(data)


e:\anaconda\envs\chatbot_new\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
import pandas as pd
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

In [4]:
model_name = 'nlpaueb/legal-bert-base-uncased'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)


e:\anaconda\envs\chatbot_new\lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at nlpaueb/legal-bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [5]:
def classify_texts(texts, tokenizer, model):
    inputs = tokenizer(texts, padding=True, truncation=True, return_tensors='pt')
    with torch.no_grad():
        outputs = model(**inputs)
    logits = outputs.logits
    predictions = torch.argmax(logits, dim=-1)
    return predictions

initial_labels = classify_texts(texts, tokenizer, model)

In [6]:
data = pd.DataFrame({'text': texts, 'label': initial_labels.numpy()})

In [7]:
data.to_csv('labeled_data2.csv', index=False)